In [77]:
!pip install pymupdf4llm

In [ ]:
!pip install -U --quiet langchain-community

In [ ]:
!pip install --quiet langchain faiss-cpu sentence-transformers requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 30.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.


In [ ]:
!pip install llama-index --no-cache-dir --quiet

# Import Modules

In [ ]:
!pip install --upgrade langchain


In [ ]:
import os
import pymupdf4llm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_classic.chains.retrieval_qa.base import RetrievalQA

# API KEY

In [ ]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("openrouter-api-key")

# Load PDF and Convert to Markdown

In [ ]:
md_read = pymupdf4llm.LlamaMarkdownReader()
data = md_read.load_data("/content/data/OPCNet Broker User Guide.pdf")
full_text = "\n".join(d.text for d in data if hasattr(d, 'text'))

# Split text into Chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs = splitter.create_documents([full_text])

print(f"Number of Chunks: {len(docs)}")

lengths = [len(d.page_content) for d in docs]
print("Min length:" , min(lengths))
print("Max length:" , max(lengths))
print("Average length:" , sum(lengths) / len(lengths))

Number of Chunks: 170
Min length: 72
Max length: 796
Average length: 664.7352941176471


# Local Embeddings + FAISS Store

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs,embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

/tmp/ipython-input-2332867890.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 # Set-up LLM API

In [ ]:
llm = ChatOpenAI(
    model="gpt-oss-120b",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.3
)

# Build RAG Chain

In [ ]:
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
from langchain_classic.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a helpful assistant that answers questions ONLY using the provided context"
        "Context: \n{context}\n\n"
        "Question: {question}\n\n"
        "If the context does not contain the necessary data to generate an answer, say 'The answer is not found in the provided document'"
        )
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

# Ask Question

In [ ]:
print("RAG Chatbot Ready!")
print("Type 'exit' to quit.\n")

while True:
    q = input("You: ").strip()
    if q.lower() in ("exit", "quit"):
        break
    result = qa_chain(q)
    print("\nAssistant:", result["result"], "\n")


RAG Chatbot Ready!
Type 'exit' to quit.

You: when onb broker is published ?


/tmp/ipython-input-55375176.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa_chain(q)



Assistant: The answer is not found in the provided document. 

You: what's ONB?

Assistant: ONB refers to **Integration Objects’ OPCNet Broker**, i.e., the OPCNet Broker server and its associated client components. It is the software that provides the OPCNet Broker service (the “ONB Server”) used for OPC communication, security, and logging. 

You: List ONB Features

Assistant: The answer is not found in the provided document. 

You: ONB Features

Assistant: The answer is not found in the provided document. 

You: System Requirements

Assistant: The answer is not found in the provided document. 

You: onb refer for what ?

Assistant: The answer is not found in the provided document. 

You: what's ONB

Assistant: ONB refers to the **OPCNet Broker** – the server component (called the “ONB Server”) that provides OPC communication services and works with ONB clients. It is the “Integration Objects OPCNet Broker Server” mentioned throughout the document. 

You: What's IO ?

Assistant: **IO